In [1]:
import openai
import json
from utils.paths import *

In [ ]:
with open(PROMPTS_PATH, "r", encoding="utf-8") as f:
    prompts = json.load(f)

system_prompt = prompts["system_prompt"]

with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    json_schema = json.load(f)

In [ ]:
def create_prompt(text):
    user_prompt = f"""
    Ниже приведён текстовый отчёт о скважине.  
    Проанализируй его и заполни все поля JSON-схемы в соответствии с данными, найденными в тексте.  
    Если какие-то данные отсутствуют — ставь пустую строку "".  
    Помни: вывод должен строго соответствовать JSON Schema и быть корректным JSON-объектом (не массивом, не текстом).

    Текст отчёта:
    {text}
    """
    return user_prompt

In [4]:
raw_text = """
Скважина №2177 расположена в Октябрьском районе. 
Эксплуатируется ООО "НефтьСервис". 
Категория — добывающая. 
Способ эксплуатации — фонтанный. 
Состояние по фонду — действующая. 
Дата последней остановки — 2024-12-15 (по причине планового обслуживания).
"""
user_prompt = create_prompt(raw_text)
user_prompt

'\n    Ниже приведён текстовый отчёт о скважине.  \n    Проанализируй его и заполни все поля JSON-схемы в соответствии с данными, найденными в тексте.  \n    Если какие-то данные отсутствуют — ставь пустую строку "".  \n    Помни: вывод должен строго соответствовать JSON Schema и быть корректным JSON-объектом (не массивом, не текстом).\n\n    Текст отчёта:\n    \nСкважина №2177 расположена в Октябрьском районе. \nЭксплуатируется ООО "НефтьСервис". \nКатегория — добывающая. \nСпособ эксплуатации — фонтанный. \nСостояние по фонду — действующая. \nДата последней остановки — 2024-12-15 (по причине планового обслуживания).\n\n    '

In [5]:
client = openai.OpenAI(
    api_key=YANDEX_CLOUD_API_KEY,
    base_url="https://llm.api.cloud.yandex.net/v1"
)

In [6]:
response = client.chat.completions.create(
    model=f"gpt://b1gdsddkrr4oalkrcfu8/qwen3-235b-a22b-fp8/latest",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=2048,
    temperature=0.0,
    stream=False,
    response_format={"type": "json_schema", "json_schema": json_schema}
)

In [7]:
answer = response.choices[0].message.content

try:
    data = json.loads(answer)
    print(json.dumps(data, indent=4, ensure_ascii=False))
except json.JSONDecodeError:
    print("⚠️ Ответ невалидный JSON, печатаю как есть:")
    print(answer)


{
    "Инвентарный номер": "2177",
    "Район": "Октябрьский",
    "Организация": "ООО \"НефтьСервис\"",
    "Категория скважины": "добывающая",
    "Способ эксплуатации": "фонтанный",
    "Состояние по фонду": "действующая",
    "Дата остановки": "2024-12-15"
}
